In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.insert(1, '../src/data')

In [6]:
import json
import re
from ast import literal_eval
from string import punctuation

import pandas as pd
from regex import UNICODE, VERBOSE, compile
from segtok.tokenizer import word_tokenizer

from main_processing import (remove_html_tags, remove_punctuation,
                                      remove_urls)
from src import PROJECT_PATHS

In [71]:
def is_punctuated(df, text_col="TEXT"):
    def punctuated(text):
        for x in text:
            if x in punctuation:
                return True
        else:
            return False
    return df[text_col].map(punctuated)

In [72]:
df = pd.read_csv(f"{PROJECT_PATHS.data}\\processed\\peeled.csv", index_col=0)

In [73]:
df.shape

(95580, 20)

In [74]:
df.head()

,TITLE,LIKES,TEXT,TAGS,COMMENTS,VIEWS,NICKNAME,TIME_GAP,TIMESTAMP,CURRENT_TIMESTAMP,politics,humour,help,children,relationships,coronavirus,poetry,job,history,ukraine
0,МВД разработало правила отправки пьяных россия...,NaN,МВД России разработало порядок доставления нет...,"['Полиция', 'Вытрезвитель', 'Опьянение', 'Прое...",50,NaN,Kanedias,2 месяца назад,2021-02-20T22:55:57+03:00,1.620672e+09,1,0,0,0,0,0,0,0,0,0
1,"Ответ на пост «""Геноцид 1933 года"" на страница...",-39.0,"А что вы хотите? Национализм и шовинизм, в том...","['[моё]', 'Политика', 'Голод', 'Голодомор', 'У...",152,Загружаем,CultDeadCow,2 месяца назад,2021-02-20T18:44:21+03:00,1.620672e+09,1,0,0,0,0,0,0,0,0,0
2,Сахалинский депутат лишился мандата из-за виде...,186.0,Депутаты Томаринского городского округа (юг Са...,"['[моё]', 'Новости', 'Политика', 'Скандал', 'С...",30,Загружаем,TASSagency,2 месяца назад,2021-02-20T12:57:10+03:00,1.620672e+09,1,0,0,0,0,0,0,0,0,0
3,Рогозин отчитался Путину об успехах «Роскосмоса»,48.0,Глава «Роскосмоса» Дмитрий Рогозин на встрече ...,"['Дмитрий Рогозин', 'Новости', 'Политика', 'Ко...",33,Загружаем,ElonMuskSpaceX,2 месяца назад,2021-02-20T12:34:19+03:00,1.620672e+09,1,0,0,0,0,0,0,0,0,0
4,Москва ждет реакцию Лондона на утечку данных о...,27.0,"МОСКВА, 20 февраля. /ТАСС/. РФ ждет реакцию Ве...","['Мария Захарова', 'Политика', 'Великобритания...",57,Загружаем,rug178,2 месяца назад,2021-02-20T12:07:30+03:00,1.620672e+09,1,0,0,0,0,0,0,0,0,0


### Split rows by target columns on strict and fuzzy classes

In [7]:
target_tags = PROJECT_PATHS.ENG_TAGS

In [76]:
df[target_tags].sum(axis=0)

politics         10005
humour            9998
help             10002
children         10002
relationships    10000
coronavirus       5572
poetry            9999
job               9998
history          10006
ukraine           9998
dtype: int64

### Remove urls

In [77]:
df["TEXT"].str.len().mean()

2495.91053567692

In [78]:
remove_urls(df)


-----------------------------------------------
            Remove urls


In [79]:
df["TEXT"].str.len().mean()

2492.611770244821

### Remove punctuation

In [80]:
df["TEXT_PUNCT"] = df["TEXT"].copy()
df["TITLE_PUNCT"] = df["TITLE"].copy()

In [81]:
remove_punctuation(df, "TEXT")


-----------------------------------------------
            Remove punctuation


In [82]:
df["TEXT"].str.len().mean()

2482.545794099184

In [83]:
df["TITLE"].str.len().mean()

33.39497802887633

In [84]:
remove_punctuation(df, 'TITLE')


-----------------------------------------------
            Remove punctuation


In [85]:
df["TITLE"].str.len().mean()

33.182684662063195

### Apply lower case

In [86]:
df["TEXT"] = df['TEXT'].str.lower()
df['TEXT_PUNCT'] = df['TEXT_PUNCT'].str.lower()

In [87]:
df.drop(columns=['VIEWS'], inplace=True)

In [88]:
df.head()

,TITLE,LIKES,TEXT,TAGS,COMMENTS,NICKNAME,TIME_GAP,TIMESTAMP,CURRENT_TIMESTAMP,politics,...,help,children,relationships,coronavirus,poetry,job,history,ukraine,TEXT_PUNCT,TITLE_PUNCT
0,МВД разработало правила отправки пьяных россия...,NaN,мвд россии разработало порядок доставления нет...,"['Полиция', 'Вытрезвитель', 'Опьянение', 'Прое...",50,Kanedias,2 месяца назад,2021-02-20T22:55:57+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,0,мвд россии разработало порядок доставления нет...,МВД разработало правила отправки пьяных россия...
1,Ответ на пост « Геноцид 1933 года на страница...,-39.0,а что вы хотите национализм и шовинизм в том...,"['[моё]', 'Политика', 'Голод', 'Голодомор', 'У...",152,CultDeadCow,2 месяца назад,2021-02-20T18:44:21+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,0,"а что вы хотите? национализм и шовинизм, в том...","Ответ на пост «""Геноцид 1933 года"" на страница..."
2,Сахалинский депутат лишился мандата из за виде...,186.0,депутаты томаринского городского округа юг са...,"['[моё]', 'Новости', 'Политика', 'Скандал', 'С...",30,TASSagency,2 месяца назад,2021-02-20T12:57:10+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,0,депутаты томаринского городского округа (юг са...,Сахалинский депутат лишился мандата из-за виде...
3,Рогозин отчитался Путину об успехах «Роскосмоса»,48.0,глава «роскосмоса» дмитрий рогозин на встрече ...,"['Дмитрий Рогозин', 'Новости', 'Политика', 'Ко...",33,ElonMuskSpaceX,2 месяца назад,2021-02-20T12:34:19+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,0,глава «роскосмоса» дмитрий рогозин на встрече ...,Рогозин отчитался Путину об успехах «Роскосмоса»
4,Москва ждет реакцию Лондона на утечку данных о...,27.0,москва 20 февраля тасс рф ждет реакцию вел...,"['Мария Захарова', 'Политика', 'Великобритания...",57,rug178,2 месяца назад,2021-02-20T12:07:30+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,0,"москва, 20 февраля. /тасс/. рф ждет реакцию ве...",Москва ждет реакцию Лондона на утечку данных о...


### Add target column

In [8]:
numered_tags = list(enumerate(sorted(target_tags)))
df['target'] = sum([df[tag].map(lambda z: x if z else 0) for x, tag in numered_tags])
numered_tags

[(0, 'children'),
 (1, 'coronavirus'),
 (2, 'help'),
 (3, 'history'),
 (4, 'humour'),
 (5, 'job'),
 (6, 'poetry'),
 (7, 'politics'),
 (8, 'relationships'),
 (9, 'ukraine')]

In [90]:
df.head()

,TITLE,LIKES,TEXT,TAGS,COMMENTS,NICKNAME,TIME_GAP,TIMESTAMP,CURRENT_TIMESTAMP,politics,...,children,relationships,coronavirus,poetry,job,history,ukraine,TEXT_PUNCT,TITLE_PUNCT,target
0,МВД разработало правила отправки пьяных россия...,NaN,мвд россии разработало порядок доставления нет...,"['Полиция', 'Вытрезвитель', 'Опьянение', 'Прое...",50,Kanedias,2 месяца назад,2021-02-20T22:55:57+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,мвд россии разработало порядок доставления нет...,МВД разработало правила отправки пьяных россия...,7
1,Ответ на пост « Геноцид 1933 года на страница...,-39.0,а что вы хотите национализм и шовинизм в том...,"['[моё]', 'Политика', 'Голод', 'Голодомор', 'У...",152,CultDeadCow,2 месяца назад,2021-02-20T18:44:21+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,"а что вы хотите? национализм и шовинизм, в том...","Ответ на пост «""Геноцид 1933 года"" на страница...",7
2,Сахалинский депутат лишился мандата из за виде...,186.0,депутаты томаринского городского округа юг са...,"['[моё]', 'Новости', 'Политика', 'Скандал', 'С...",30,TASSagency,2 месяца назад,2021-02-20T12:57:10+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,депутаты томаринского городского округа (юг са...,Сахалинский депутат лишился мандата из-за виде...,7
3,Рогозин отчитался Путину об успехах «Роскосмоса»,48.0,глава «роскосмоса» дмитрий рогозин на встрече ...,"['Дмитрий Рогозин', 'Новости', 'Политика', 'Ко...",33,ElonMuskSpaceX,2 месяца назад,2021-02-20T12:34:19+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,глава «роскосмоса» дмитрий рогозин на встрече ...,Рогозин отчитался Путину об успехах «Роскосмоса»,7
4,Москва ждет реакцию Лондона на утечку данных о...,27.0,москва 20 февраля тасс рф ждет реакцию вел...,"['Мария Захарова', 'Политика', 'Великобритания...",57,rug178,2 месяца назад,2021-02-20T12:07:30+03:00,1.620672e+09,1,...,0,0,0,0,0,0,0,"москва, 20 февраля. /тасс/. рф ждет реакцию ве...",Москва ждет реакцию Лондона на утечку данных о...,7


In [91]:
df[target_tags].sum(axis=0)

politics         10005
humour            9998
help             10002
children         10002
relationships    10000
coronavirus       5572
poetry            9999
job               9998
history          10006
ukraine           9998
dtype: int64

In [92]:
df.groupby('target')['target'].count()

target
0    10002
1     5572
2    10002
3    10006
4     9998
5     9998
6     9999
7    10005
8    10000
9     9998
Name: target, dtype: int64

In [93]:
for x in target_tags:
    del df[x]

In [94]:
df.head()

,TITLE,LIKES,TEXT,TAGS,COMMENTS,NICKNAME,TIME_GAP,TIMESTAMP,CURRENT_TIMESTAMP,TEXT_PUNCT,TITLE_PUNCT,target
0,МВД разработало правила отправки пьяных россия...,NaN,мвд россии разработало порядок доставления нет...,"['Полиция', 'Вытрезвитель', 'Опьянение', 'Прое...",50,Kanedias,2 месяца назад,2021-02-20T22:55:57+03:00,1.620672e+09,мвд россии разработало порядок доставления нет...,МВД разработало правила отправки пьяных россия...,7
1,Ответ на пост « Геноцид 1933 года на страница...,-39.0,а что вы хотите национализм и шовинизм в том...,"['[моё]', 'Политика', 'Голод', 'Голодомор', 'У...",152,CultDeadCow,2 месяца назад,2021-02-20T18:44:21+03:00,1.620672e+09,"а что вы хотите? национализм и шовинизм, в том...","Ответ на пост «""Геноцид 1933 года"" на страница...",7
2,Сахалинский депутат лишился мандата из за виде...,186.0,депутаты томаринского городского округа юг са...,"['[моё]', 'Новости', 'Политика', 'Скандал', 'С...",30,TASSagency,2 месяца назад,2021-02-20T12:57:10+03:00,1.620672e+09,депутаты томаринского городского округа (юг са...,Сахалинский депутат лишился мандата из-за виде...,7
3,Рогозин отчитался Путину об успехах «Роскосмоса»,48.0,глава «роскосмоса» дмитрий рогозин на встрече ...,"['Дмитрий Рогозин', 'Новости', 'Политика', 'Ко...",33,ElonMuskSpaceX,2 месяца назад,2021-02-20T12:34:19+03:00,1.620672e+09,глава «роскосмоса» дмитрий рогозин на встрече ...,Рогозин отчитался Путину об успехах «Роскосмоса»,7
4,Москва ждет реакцию Лондона на утечку данных о...,27.0,москва 20 февраля тасс рф ждет реакцию вел...,"['Мария Захарова', 'Политика', 'Великобритания...",57,rug178,2 месяца назад,2021-02-20T12:07:30+03:00,1.620672e+09,"москва, 20 февраля. /тасс/. рф ждет реакцию ве...",Москва ждет реакцию Лондона на утечку данных о...,7


In [96]:
df.to_csv(f"{PROJECT_PATHS.data}\\processed\\preprocessed.csv")